<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Logging the payload with scoring endpoint wrapper

Now, you're ready to create credit risk scoring endpoint wrapper. The goal of the wrapper is to automatically log all input and outpus payloads from credit risk deployment (created previously).

## Prerequisites to this tutorial:
> * Credit Risk model deployment on Azure ML Service (scoring endpoint)
> * Custom ML Engine deployment and integration with OpenScale


In this part of the tutorial, you use Azure Machine Learning service to:

> * Set up your testing environment
> * Retrieve the deployment from your workspace
> * Test the deployment locally
> * Deploy the wrapper to ACI
> * Test the wrapper

## Install `ibm-ai-openscale` package

In [1]:
!pip install --upgrade ibm-ai-openscale | tail -n 1

You should consider upgrading via the 'pip install --upgrade pip' command.


### Action: Restart the kernel

## Set up the workspace

In [1]:
import os
import urllib.request
from azureml.core import Workspace

ws = Workspace.from_config()

In [2]:
ws.get_details()

{'id': '/subscriptions/744bca72-2299-451c-b682-ed6fb75fb671/resourceGroups/ai-ops-squad/providers/Microsoft.MachineLearningServices/workspaces/ai-ops-squad-server',
 'name': 'ai-ops-squad-server',
 'location': 'southcentralus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'workspaceid': '16b06534-668c-4887-aa3c-a10d99eb27e4',
 'description': '',
 'friendlyName': '',
 'creationTime': '2018-10-18T12:12:25.7036291+00:00',
 'containerRegistry': '/subscriptions/744bca72-2299-451c-b682-ed6fb75fb671/resourcegroups/ai-ops-squad/providers/microsoft.containerregistry/registries/aiopssquadserv4124490709',
 'keyVault': '/subscriptions/744bca72-2299-451c-b682-ed6fb75fb671/resourcegroups/ai-ops-squad/providers/microsoft.keyvault/vaults/aiopssquadserv3633962467',
 'applicationInsights': '/subscriptions/744bca72-2299-451c-b682-ed6fb75fb671/resourcegroups/ai-ops-squad/providers/microsoft.insights/components/aiopssquadserv0294391293',
 'identityPrincipalId': 'e057536e-7de0-4980-8a09-eabf175

## Set up the environment

Start by setting up a testing environment.

### Import packages

Import the Python packages needed for this tutorial.

In [3]:
import azureml.core
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.supporting_classes import PayloadRecord
from ibm_ai_openscale.utils import *
import os 

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.33


### Retrieve deployments

You created a model deployment in your workspace in the previous tutorial. Now, let's list the deployments.

In [4]:
from azureml.core.webservice import AciWebservice

credit_risk_deployment_name = 'credit-risk-prediction'
credit_risk_scoring_endpoint = None

webservices = AciWebservice.list(ws)
for service in webservices:
    if service.name == credit_risk_deployment_name:
        credit_risk_scoring_endpoint = service.scoring_uri
        
print('scoring endpoint', credit_risk_scoring_endpoint)

scoring endpoint http://20.189.138.213:80/score


## Test endpoint locally

Before creating a scoring endpoint wrapper let's test the original scoring endpoint.

In [5]:
scoring_data = {"input":[{
                            'CheckingStatus': "0_to_200", 'LoanDuration': 31, 'CreditHistory': "credits_paid_to_date", 'LoanPurpose': "other",
                            'LoanAmount': 1889, 'ExistingSavings': "100_to_500",'EmploymentDuration': "less_1",'InstallmentPercent': 3,'Sex': "female",
                            'OthersOnLoan': "none",'CurrentResidenceDuration': 3, 'OwnsProperty': "savings_insurance", 'Age': 32,'InstallmentPlans': "none",
                            'Housing': "own",'ExistingCreditsCount': 1,'Job': "skilled",'Dependents': 1,'Telephone': "none",'ForeignWorker': "yes",
                        },
                        {
                            'CheckingStatus': "no_checking", 'LoanDuration': 13, 'CreditHistory': "credits_paid_to_date", 'LoanPurpose': "car_new",
                            'LoanAmount': 1389, 'ExistingSavings': "100_to_500",'EmploymentDuration': "1_to_4",'InstallmentPercent': 2,'Sex': "male",
                            'OthersOnLoan': "none",'CurrentResidenceDuration': 3, 'OwnsProperty': "savings_insurance", 'Age': 25,'InstallmentPlans': "none",
                            'Housing': "own",'ExistingCreditsCount': 2,'Job': "skilled",'Dependents': 2,'Telephone': "none",'ForeignWorker': "yes",
                        }]
              }

In [6]:
import requests

headers = {'Content-Type':'application/json'}
resp = requests.post(credit_risk_scoring_endpoint, json=scoring_data, headers=headers)
output_data = json.loads(resp.json())

print("POST to url", credit_risk_scoring_endpoint)
print(output_data)

POST to url http://20.189.138.213:80/score
{'output': [{'Scored Labels': 'No Risk', 'Scored Probabilities': [0.8922524675865824, 0.10774753241341757]}, {'Scored Labels': 'No Risk', 'Scored Probabilities': [0.8335192848546905, 0.1664807151453095]}]}


## Implement the wrapper

Once you've tested the deployment, let's implement the wrapper that will:
* call the original scoring endpoint (credit risk)
* conver the scoring request and response to the OpenScale format
* store converted request and response as payload records in OpenScale data mart


To build the correct environment for ACI, provide the following:
* A scoring script to show how to use the scoring wrapper
* An environment file to show what packages need to be installed
* A configuration file to build the ACI


### Create scoring script

Create the scoring script, called score.py, used by the web service call to show how to use the deployment.

You must include two required functions into the scoring script:
* The `init()` function, which typically loads the required credentials into a global object. This function is run only once when the Docker container is started. 

* The `run(input_data)` function uses the original scoring endpoint to predict a value based on the input data. Next it makes format conversions and finally stored payload records in OpenScale data mart.


### ACTION: You need to update below score.py content by changing the following lines:

- `scoring_endpoint` - PUT your scoring endpoint there
- `scoring_headers` - PUT your scoring header there
- `openscale_credentials` - PUT your OpenScale credentials there
- `openscale_subscription_uid` - PUT uid of created subscription there
- you may also need to modify conversion methods to fit your custom format of payloads

### Scoring endpoint wrapper code

In [7]:
%%writefile score.py
import json
import numpy as np
import os
import time
import requests
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.supporting_classes import PayloadRecord


def convert_user_input_2_openscale(input_data):
    users_records = input_data['input']
    openscale_fields = list(users_records[0])
    openscale_values = [[rec[k] for k in openscale_fields] for rec in users_records] 

    return {'fields':openscale_fields, 'values':openscale_values}


def convert_user_output_2_openscale(output_data):
    output_data = json.loads(output_data)
    users_records = output_data['output']
    openscale_fields = list(users_records[0])
    openscale_values = [[rec[k] for k in openscale_fields] for rec in users_records] 

    return {'fields':openscale_fields, 'values':openscale_values}


def init():
    global openscale_client
    global openscale_credentials
    global openscale_subscription_uid
    global openscale_subscription
    global scoring_endpoint
    global scoring_headers
    
    scoring_endpoint = <PUT YOUR ENDPOINT URL HERE>
    scoring_headers = {'Content-Type': 'application/json'}
    openscale_credentials = {<PUT YOUR OpenScale CREDENTIALS HERE>}
    openscale_subscription_uid = <PUT YOUR SUBSCRIPTION ID HERE>
    openscale_client = APIClient(openscale_credentials)
    openscale_subscription = openscale_client.data_mart.subscriptions.get(openscale_subscription_uid)
    
    
def run(input_data):
    try:
        # ------ CALL SCORING ENDPOINT --------------
        if type(input_data) is str:
            input_data = json.loads(input_data)
        
        start_time = time.time()        
        response = requests.post(scoring_endpoint, json=input_data, headers=scoring_headers)
        response_time = int((time.time() - start_time)*1000)
        output_data = response.json()
        

        # ------ PAYLOAD COVERSION TO OPENSCALE FORMAT and LOGGING --------------
        openscale_input = convert_user_input_2_openscale(input_data)
        openscale_output = convert_user_output_2_openscale(output_data)
        
        records_list = [PayloadRecord(request=openscale_input, response=openscale_output, response_time=response_time)]
        openscale_subscription.payload_logging.store(records=records_list)       
        
        return output_data
        
    except Exception as e:
        error = str(e)
        return json.dumps({"error": error + str(input_data)})
        

Overwriting score.py


## Test wrapper locally

In [14]:
from score import *

init()

In [9]:
scores = run(scoring_data)

In [10]:
json.loads(scores)

{'output': [{'Scored Labels': 'No Risk',
   'Scored Probabilities': [0.8922524675865824, 0.10774753241341757]},
  {'Scored Labels': 'No Risk',
   'Scored Probabilities': [0.8335192848546905, 0.1664807151453095]}]}

In [15]:
latest_record = openscale_subscription.payload_logging.get_table_content(limit=1)
latest_record

,scoring_id,scoring_timestamp,deployment_id,asset_revision,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,...,Job,Dependents,Telephone,ForeignWorker,Scored Labels,Scored Probabilities,prediction_probability,debiased_prediction,debiased_probability,debiased_decoded_target
0,4ea8ca46-185f-4e92-a151-a1bf1ca0f6c3-1,2019-05-17 09:56:20.310000+00:00,credit,None,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,...,skilled,1,none,yes,No Risk,"[0.8922524675865824, 0.10774753241341757]",0.892252,None,None,None


## Deploy the wrapper as web service

### Create environment file

Next, create an environment file, called myenv.yml, that specifies all of the script's package dependencies. This file is used to ensure that all of those dependencies are installed in the Docker image. This model needs `scikit-learn` and `azureml-sdk`.

In [16]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_pip_package("ibm-ai-openscale")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

Review the content of the `myenv.yml` file.

In [17]:
with open("myenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults
  - ibm-ai-openscale



### Create configuration file

Create a deployment configuration file and specify the number of CPUs and gigabyte of RAM needed for your ACI container. While it depends on your model, the default of 1 core and 1 gigabyte of RAM is usually sufficient for many models. If you feel you need more later, you would have to recreate the image and redeploy the service.

In [18]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "german credit risk",  "method" : "scoring-endpoint-wrapper"}, 
                                               description='Credit risk scoring endpoint with payload logging')

### Deploy in ACI
Estimated time to complete: **about 7-8 minutes**

Configure the image and deploy. The following code goes through these steps:

1. Build an image using:
   * The scoring file (`score.py`)
   * The environment file (`myenv.yml`)
   * The model file
1. Register that image under the workspace. 
1. Send the image to the ACI container.
1. Start up a container in ACI using the image.
1. Get the web service HTTP endpoint.

## Scoring endpoint creation

In [19]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage


deployment_name = 'credit-risk-prediction-wrapper'

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml")

service_az = Webservice.deploy_from_model(workspace=ws,
                                       name=deployment_name,
                                       deployment_config=aciconfig,
                                       models=[],
                                       image_config=image_config)

service_az.wait_for_deployment(show_output=True)

Creating image
Image creation operation finished for image credit-risk-prediction-wrapper:6, operation "Succeeded"
Creating service
Running...............
SucceededACI service creation operation finished, operation "Succeeded"
CPU times: user 989 ms, sys: 386 ms, total: 1.38 s
Wall time: 3min 22s


Get the scoring web service's HTTP endpoint, which accepts REST client calls. This endpoint can be shared with anyone who wants to test the web service or integrate it into an application.

In [20]:
print(service_az.scoring_uri)

http://13.86.233.123:80/score


## Test deployed wrapper

The following code goes through these steps:
1. Send the scoring request to the web service hosted in ACI. 
2. Print the returned predictions.
3. Check if the scoring request and response has been logged as payload records in OpenScale

In [27]:
import requests

headers = {'Content-Type':'application/json'}
resp = requests.post(service_az.scoring_uri, json=scoring_data, headers=headers)

print("POST to url", service_az.scoring_uri)
print(resp.json())

POST to url http://13.86.233.123:80/score
{"output": [{"Scored Labels": "No Risk", "Scored Probabilities": [0.8922524675865824, 0.10774753241341757]}, {"Scored Labels": "No Risk", "Scored Probabilities": [0.8335192848546905, 0.1664807151453095]}]}


In [29]:
time.sleep(4)

latest_record = openscale_subscription.payload_logging.get_table_content(limit=1)
latest_record

,scoring_id,scoring_timestamp,deployment_id,asset_revision,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,...,Job,Dependents,Telephone,ForeignWorker,Scored Labels,Scored Probabilities,prediction_probability,debiased_prediction,debiased_probability,debiased_decoded_target
0,fbf6ef96-99e2-434e-bc27-806ed419a521-1,2019-05-17 10:02:56.367000+00:00,credit,None,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,...,skilled,1,none,yes,No Risk,"[0.8922524675865824, 0.10774753241341757]",0.892252,None,None,None
